In [ ]:
# Colab cell 1: install requirements
!pip install flask==2.3.2 opencv-python-headless==4.8.0.76 numpy==1.26.0 pillow==10.1.0 pyngrok==6.0.0

In [ ]:
# Colab cell 2: create directories and files that the Flask app will use
import os, textwrap, pathlib, sqlite3, secrets

BASE_DIR = "/content/image_defect_app"
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, "uploads"), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, "processed"), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, "templates"), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, "static"), exist_ok=True)

print("App base directory:", BASE_DIR)


App base directory: /content/image_defect_app


In [ ]:
# Colab cell 3: write Flask app code to file
app_code = r'''
from flask import Flask, render_template, request, redirect, url_for, session, send_from_directory, flash
import os, sqlite3, uuid, datetime
from werkzeug.utils import secure_filename
from werkzeug.security import generate_password_hash, check_password_hash
import cv2
import numpy as np

BASE_DIR = os.path.abspath(os.path.dirname(__file__))
UPLOAD_FOLDER = os.path.join(BASE_DIR, "uploads")
PROCESSED_FOLDER = os.path.join(BASE_DIR, "processed")
DB_PATH = os.path.join(BASE_DIR, "app.db")

ALLOWED_EXTENSIONS = {"png","jpg","jpeg"}

app = Flask(__name__, static_folder="static", template_folder="templates")
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['PROCESSED_FOLDER'] = PROCESSED_FOLDER
app.secret_key = os.environ.get("FLASK_SECRET_KEY", "dev_secret_please_change")

# -------------------------
# Database helpers
# -------------------------
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    created_at TEXT NOT NULL
)
""")
    cur.execute("""
CREATE TABLE IF NOT EXISTS images (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    filename TEXT NOT NULL,
    processed_filename TEXT,
    defect_count INTEGER DEFAULT 0,
    uploaded_at TEXT,
    FOREIGN KEY(user_id) REFERENCES users(id)
)
""")
    conn.commit()
    conn.close()

def get_db_conn():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

# -------------------------
# User auth helpers
# -------------------------
def register_user(email, password):
    if len(password) < 6:
        return False, "Password must be at least 6 characters."
    pw_hash = generate_password_hash(password)
    conn = get_db_db_conn()
    try:
        conn.execute("INSERT INTO users (email, password_hash, created_at) VALUES (?, ?, ?)",
                     (email, pw_hash, datetime.datetime.utcnow().isoformat()))
        conn.commit()
    except Exception as e:
        conn.close()
        return False, "Email already registered or DB error."
    conn.close()
    return True, "Registered"

def verify_user(email, password):
    conn = get_db_conn()
    row = conn.execute("SELECT * FROM users WHERE email = ?", (email,)).fetchone()
    conn.close()
    if not row:
        return False, "Email not found."
    if check_password_hash(row["password_hash"], password):
        return True, row["id"]
    else:
        return False, "Incorrect password."

# -------------------------
# Utilities
# -------------------------
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS

# Basic defect detection pipeline using OpenCV
def detect_defects(input_path, output_path, min_contour_area=100):
    # Read and preprocess
    img = cv2.imdecode(np.fromfile(input_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError("Cannot read input image.")
    orig = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Denoise
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    # Edge detection (Canny)
    edges = cv2.Canny(blur, 50, 150)
    # Dilate edges to close gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilated = cv2.dilate(edges, kernel, iterations=1)
    cnts, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter and draw bounding boxes on a copy
    boxed = orig.copy()
    defect_count = 0
    for c in cnts:
        area = cv2.contourArea(c)
        if area < min_contour_area:
            continue
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(boxed, (x,y), (x+w,y+h), (0,0,255), 2)
        defect_count += 1
    # Save processed image
    # For windows filesystem compatibility, use cv2.imencode then tofile
    ext = os.path.splitext(output_path)[1][1:]
    if ext == '':
        ext = 'jpg'
    is_success, buffer = cv2.imencode('.' + ext, boxed)
    if not is_success:
        raise ValueError("Failed to encode processed image.")
    buffer.tofile(output_path)
    return defect_count, output_path

# -------------------------
# Routes
# -------------------------
@app.route("/")
def index():
    user = None
    if 'user_id' in session:
        conn = get_db_conn()
        user = conn.execute("SELECT id, email FROM users WHERE id = ?", (session['user_id'],)).fetchone()
        conn.close()
    return render_template("index.html", user=user)

@app.route("/register", methods=["GET","POST"])
def register():
    if request.method == "POST":
        email = request.form.get("email","").strip().lower()
        password = request.form.get("password","")
        ok, msg = register_user(email, password)
        if ok:
            flash("Registration successful. Please login.")
            return redirect(url_for("login"))
        else:
            flash(msg)
            return redirect(url_for("register"))
    return render_template("register.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        email = request.form.get("email","").strip().lower()
        password = request.form.get("password","")
        ok, data = verify_user(email, password)
        if ok:
            session['user_id'] = data
            return redirect(url_for("dashboard"))
        else:
            flash(data)
            return redirect(url_for("login"))
    return render_template("login.html")

@app.route("/logout")
def logout():
    session.pop('user_id', None)
    flash("Logged out.")
    return redirect(url_for("index"))

@app.route("/dashboard")
def dashboard():
    if 'user_id' not in session:
        return redirect(url_for("login"))
    conn = get_db_conn()
    images = conn.execute("SELECT * FROM images WHERE user_id = ? ORDER BY uploaded_at DESC", (session['user_id'],)).fetchall()
    conn.close()
    return render_template("dashboard.html", images=images)

@app.route("/upload", methods=["GET","POST"])
def upload():
    if 'user_id' not in session:
        return redirect(url_for("login"))
    if request.method == "POST":
        if 'image' not in request.files:
            flash("No file part.")
            return redirect(request.url)
        file = request.files['image']
        if file.filename == "":
            flash("No selected file.")
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            unique_id = str(uuid.uuid4())[:8]
            save_name = f"{unique_id}_{filename}"
            save_path = os.path.join(app.config['UPLOAD_FOLDER'], save_name)
            # Use file.save directly
            file.save(save_path)
            # Process
            processed_name = f"proc_{save_name}"
            processed_path = os.path.join(app.config['PROCESSED_FOLDER'], processed_name)
            try:
                defect_count, out_path = detect_defects(save_path, processed_path, min_contour_area=150)
            except Exception as e:
                flash("Processing failed: " + str(e))
                return redirect(request.url)
            # Save record
            conn = get_db_conn()
            conn.execute("INSERT INTO images (user_id, filename, processed_filename, defect_count, uploaded_at) VALUES (?, ?, ?, ?, ?)",
                         (session['user_id'], save_name, processed_name, defect_count, datetime.datetime.utcnow().isoformat()))
            conn.commit()
            conn.close()
            flash(f"Uploaded and processed. Potential defects found: {defect_count}")
            return redirect(url_for("dashboard"))
        else:
            flash("Invalid file type.")
            return redirect(request.url)
    return render_template("upload.html")

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route("/processed/<filename>")
def processed_file(filename):
    return send_from_directory(app.config['PROCESSED_FOLDER'], filename)

@app.route("/report/<int:image_id>")
def report(image_id):
    if 'user_id' not in session:
        return redirect(url_for("login"))
    conn = get_db_conn()
    row = conn.execute("SELECT * FROM images WHERE id = ? AND user_id = ?", (image_id, session['user_id'])).fetchone()
    conn.close()
    if not row:
        flash("Report not found.")
        return redirect(url_for("dashboard"))
    return render_template("report.html", image=row)

# -------------------------
# Start / Init
# -------------------------
if __name__ == "__main__":
    init_db()
    app.run(host="0.0.0.0", port=5000, debug=True)
'''
open(os.path.join(BASE_DIR, "app.py"), "w").write(app_code)
print("Wrote app.py to", os.path.join(BASE_DIR, "app.py"))

Wrote app.py to /content/image_defect_app/app.py


Next, let's create the static directory and add a CSS file for styling.

In [ ]:
# Colab cell 5: write CSS file
css_code = """
body {
    font-family: sans-serif;
    line-height: 1.6;
    margin: 20px;
    background-color: #f4f4f4;
}

.container {
    max-width: 800px;
    margin: auto;
    background: #fff;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}

h1, h2 {
    color: #333;
}

a {
    color: #3498db;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

form label {
    display: block;
    margin-bottom: 5px;
    font-weight: bold;
}

form input[type="email"],
form input[type="password"],
form input[type="file"] {
    width: 100%;
    padding: 8px;
    margin-bottom: 10px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

form input[type="submit"] {
    background-color: #5cb85c;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 16px;
}

form input[type="submit"]:hover {
    background-color: #4cae4c;
}

.flashes {
    list-style: none;
    padding: 0;
    margin-bottom: 15px;
}

.flashes li {
    background-color: #f2dede;
    color: #a94442;
    border: 1px solid #ebccd1;
    padding: 10px;
    margin-bottom: 5px;
    border-radius: 4px;
}

table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
}

table, th, td {
    border: 1px solid #ddd;
}

th, td {
    padding: 10px;
    text-align: left;
}

th {
    background-color: #f2f2f2;
}

img {
    max-width: 100%;
    height: auto;
    display: block;
    margin: 10px 0;
    border: 1px solid #ddd;
    border-radius: 4px;
}

"""

# Ensure the static directory exists (already created in cell 2, but good practice to check)
os.makedirs(os.path.join(BASE_DIR, "static"), exist_ok=True)

with open(os.path.join(BASE_DIR, "static", "style.css"), "w") as f:
    f.write(css_code)

print("Wrote style.css to", os.path.join(BASE_DIR, "static", "style.css"))

Wrote style.css to /content/image_defect_app/static/style.css


Next, let's create the HTML template files for the Flask application.

Finally, let's run the Flask application using `pyngrok` to create a public URL. You will need to click the generated link to access the application.

In [ ]:
# Colab cell 4: write HTML templates
html_templates = {
    "index.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Image Defect Detection App</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Welcome to the Image Defect Detection App</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        {% if user %}
            <p>Logged in as {{ user.email }}.</p>
            <p><a href="{{ url_for('dashboard') }}">Go to Dashboard</a></p>
            <p><a href="{{ url_for('logout') }}">Logout</a></p>
        {% else %}
            <p>Please <a href="{{ url_for('login') }}">Login</a> or <a href="{{ url_for('register') }}">Register</a>.</p>
        {% endif %}
    </div>
</body>
</html>
""",
    "register.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Register</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Register</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        <form method="post">
            <label for="email">Email:</label><br>
            <input type="email" id="email" name="email" required><br><br>
            <label for="password">Password:</label><br>
            <input type="password" id="password" name="password" required><br><br>
            <input type="submit" value="Register">
        </form>
        <p>Already have an account? <a href="{{ url_for('login') }}">Login here</a>.</p>
    </div>
</body>
</html>
""",
    "login.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Login</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Login</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        <form method="post">
            <label for="email">Email:</label><br>
            <input type="email" id="email" name="email" required><br><br>
            <label for="password">Password:</label><br>
            <input type="password" id="password" name="password" required><br><br>
            <input type="submit" value="Login">
        </form>
        <p>Don't have an account? <a href="{{ url_for('register') }}">Register here</a>.</p>
    </div>
</body>
</html>
""",
    "dashboard.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Dashboard</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Dashboard</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        <p><a href="{{ url_for('upload') }}">Upload New Image</a></p>
        <h2>Your Uploaded Images</h2>
        {% if images %}
            <table>
                <thead>
                    <tr>
                        <th>Filename</th>
                        <th>Uploaded At</th>
                        <th>Potential Defects</th>
                        <th>Report</th>
                    </tr>
                </thead>
                <tbody>
                    {% for image in images %}
                    <tr>
                        <td>{{ image.filename }}</td>
                        <td>{{ image.uploaded_at }}</td>
                        <td>{{ image.defect_count }}</td>
                        <td><a href="{{ url_for('report', image_id=image.id) }}">View Report</a></td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% else %}
            <p>No images uploaded yet.</p>
        {% endif %}
        <p><a href="{{ url_for('logout') }}">Logout</a></p>
    </div>
</body>
</html>
""",
    "upload.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Upload Image</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Upload Image for Defect Detection</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        <form method="post" enctype="multipart/form-data">
            <label for="image">Select Image:</label><br>
            <input type="file" id="image" name="image" accept=".png,.jpg,.jpeg" required><br><br>
            <input type="submit" value="Upload and Process">
        </form>
        <p><a href="{{ url_for('dashboard') }}">Back to Dashboard</a></p>
    </div>
</body>
</html>
""",
    "report.html": """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Image Report</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>Image Report</h1>
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                <ul class="flashes">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        {% endwith %}
        {% if image %}
            <h2>Original Image</h2>
            <img src="{{ url_for('uploaded_file', filename=image.filename) }}" alt="Original Image" width="400">
            <h2>Processed Image</h2>
            {% if image.processed_filename %}
                <img src="{{ url_for('processed_file', filename=image.processed_filename) }}" alt="Processed Image" width="400">
            {% else %}
                <p>Processed image not available.</p>
            {% endif %}
            <p><strong>Filename:</strong> {{ image.filename }}</p>
            <p><strong>Uploaded At:</strong> {{ image.uploaded_at }}</p>
            <p><strong>Potential Defects Found:</strong> {{ image.defect_count }}</p>
        {% else %}
            <p>Image report not found.</p>
        {% endif %}
        <p><a href="{{ url_for('dashboard') }}">Back to Dashboard</a></p>
    </div>
</body>
</html>
"""
}

for filename, content in html_templates.items():
    with open(os.path.join(BASE_DIR, "templates", filename), "w") as f:
        f.write(textwrap.dedent(content))
    print(f"Wrote {filename} to", os.path.join(BASE_DIR, "templates", filename))

Wrote index.html to /content/image_defect_app/templates/index.html
Wrote register.html to /content/image_defect_app/templates/register.html
Wrote login.html to /content/image_defect_app/templates/login.html
Wrote dashboard.html to /content/image_defect_app/templates/dashboard.html
Wrote upload.html to /content/image_defect_app/templates/upload.html
Wrote report.html to /content/image_defect_app/templates/report.html


In [ ]:
# Colab cell 4: write template files
BASE = "/content/image_defect_app"
T = lambda p, s: open(os.path.join(BASE, "templates", p), "w").write(s)

T("base.html", """
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>Image Defect Detection - Prototype</title>
  <style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    nav { margin-bottom: 20px; }
    .card { border: 1px solid #ddd; padding: 10px; border-radius: 6px; margin-bottom: 10px; }
    .row { display:flex; gap:12px; align-items:flex-start; }
    .col { flex:1; }
    .thumb { max-width: 320px; max-height: 240px; }
    .flash { background:#fff3cd; padding:10px; border-radius:6px; margin-bottom:10px; }
  </style>
</head>
<body>
<nav>
  <a href="{{ url_for('index') }}">Home</a> |
  {% if session.user_id %}
    <a href="{{ url_for('dashboard') }}">Dashboard</a> |
    <a href="{{ url_for('upload') }}">Upload Image</a> |
    <a href="{{ url_for('logout') }}">Logout</a>
  {% else %}
    <a href="{{ url_for('login') }}">Login</a> |
    <a href="{{ url_for('register') }}">Register</a>
  {% endif %}
</nav>

{% with messages = get_flashed_messages() %}
  {% if messages %}
    {% for m in messages %}
      <div class="flash">{{ m }}</div>
    {% endfor %}
  {% endif %}
{% endwith %}

{% block content %}{% endblock %}
</body>
</html>
""")

T("index.html", """
{% extends "base.html" %}
{% block content %}
  <h2>Welcome to Image Defect Detection (Prototype)</h2>
  <p>This prototype allows users to register, upload images, and run a classical image processing pipeline to mark potential defects.</p>
  {% if session.user_id %}
    <p><a href="{{ url_for('dashboard') }}">Go to your dashboard</a></p>
  {% else %}
    <p><a href="{{ url_for('register') }}">Register</a> or <a href="{{ url_for('login') }}">Login</a> to start.</p>
  {% endif %}
{% endblock %}
""")

T("register.html", """
{% extends "base.html" %}
{% block content %}
<h3>Register</h3>
<form method="post">
  <label>Email<br><input name="email" type="email" required></label><br><br>
  <label>Password (min 6 chars)<br><input name="password" type="password" required></label><br><br>
  <button type="submit">Register</button>
</form>
{% endblock %}
""")

T("login.html", """
{% extends "base.html" %}
{% block content %}
<h3>Login</h3>
<form method="post">
  <label>Email<br><input name="email" type="email" required></label><br><br>
  <label>Password<br><input name="password" type="password" required></label><br><br>
  <button type="submit">Login</button>
</form>
{% endblock %}
""")

T("dashboard.html", """
{% extends "base.html" %}
{% block content %}
<h3>Your Dashboard</h3>
<p><a href="{{ url_for('upload') }}">Upload a new image</a></p>
{% if images|length == 0 %}
  <p>No uploads yet.</p>
{% else %}
  {% for img in images %}
    <div class="card">
      <div class="row">
        <div class="col" style="flex:0 0 340px">
          <img src="{{ url_for('uploaded_file', filename=img['filename']) }}" class="thumb"><br>
          <small>Original</small>
        </div>
        <div class="col" style="flex:0 0 340px">
          <img src="{{ url_for('processed_file', filename=img['processed_filename']) }}" class="thumb"><br>
          <small>Processed (defects highlighted)</small>
        </div>
        <div class="col">
          <p><strong>Defects Detected:</strong> {{ img['defect_count'] }}</p>
          <p><strong>Uploaded:</strong> {{ img['uploaded_at'] }}</p>
          <p><a href="{{ url_for('report', image_id=img['id']) }}">View Report</a></p>
        </div>
      </div>
    </div>
  {% endfor %}
{% endif %}
{% endblock %}
""")

T("upload.html", """
{% extends "base.html" %}
{% block content %}
<h3>Upload Image</h3>
<form method="post" enctype="multipart/form-data">
  <input type="file" name="image" accept=".png,.jpg,.jpeg" required><br><br>
  <button type="submit">Upload & Process</button>
</form>
{% endblock %}
""")

T("report.html", """
{% extends "base.html" %}
{% block content %}
<h3>Simple Report</h3>
<div class="card">
  <div class="row">
    <div class="col">
      <img src="{{ url_for('uploaded_file', filename=image['filename']) }}" class="thumb"><br><small>Original</small>
    </div>
    <div class="col">
      <img src="{{ url_for('processed_file', filename=image['processed_filename']) }}" class="thumb"><br><small>Processed</small>
    </div>
    <div class="col">
      <p><strong>Defect Count:</strong> {{ image['defect_count'] }}</p>
      <p><strong>Uploaded At:</strong> {{ image['uploaded_at'] }}</p>
      <p><a href="{{ url_for('dashboard') }}">Back to Dashboard</a></p>
    </div>
  </div>
</div>
{% endblock %}
""")

print("Templates created.")


Templates created.


In [ ]:
# Colab cell 5: create uploads/processed dirs (already done), and helper run script to start Flask with ngrok
BASE = "/content/image_defect_app"
print("Base dir:", BASE)
!ls -la /content/image_defect_app


Base dir: /content/image_defect_app
total 36
drwxr-xr-x 6 root root 4096 Sep 12 06:20 .
drwxr-xr-x 1 root root 4096 Sep 12 05:56 ..
-rw-r--r-- 1 root root 8752 Sep 12 06:23 app.py
drwxr-xr-x 2 root root 4096 Sep 12 05:56 processed
drwxr-xr-x 2 root root 4096 Sep 12 06:22 static
drwxr-xr-x 2 root root 4096 Sep 12 06:30 templates
drwxr-xr-x 2 root root 4096 Sep 12 05:56 uploads


In [35]:
# Colab cell 6: start the flask app with pyngrok (run when ready)
from pyngrok import ngrok, conf
import subprocess, time, os, signal
from google.colab import userdata # Import userdata to access secrets

BASE = "/content/image_defect_app"
app_py = os.path.join(BASE, "app.py")

# Ensure DB init before launching server
# (the app's __main__ init_db() will run if started directly, but init here for clarity)
import sqlite3
DB_PATH = os.path.join(BASE, "app.db")
conn = sqlite3.connect(DB_PATH)
conn.close()

# Kill any existing ngrok tunnels (helpful if re-running)
try:
    ngrok.kill()
except:
    pass

# Set the ngrok auth token from Colab secrets if available
# Replace 'NGROK_AUTH_TOKEN' with the name of your secret
# If you don't have an ngrok auth token, get one from https://dashboard.ngrok.com/get-started/your-authtoken
# and add it to Colab secrets (click the key icon in the left sidebar).
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
if NGROK_AUTH_TOKEN:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
else:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. ngrok may not work.")


public_url = ngrok.connect(5000).public_url
print(" * Ngrok URL:", public_url)
print(" * Starting Flask server... (this will run in the background in this cell)")

# Start Flask using subprocess; keep it attached so output shows in the cell
# Use env to set FLASK_APP and avoid reloader for subprocess
env = os.environ.copy()
env["FLASK_APP"] = app_py
env["FLASK_ENV"] = "development"
env["PYTHONUNBUFFERED"] = "1"
# Set the FLASK_SECRET_KEY for session management
env["FLASK_SECRET_KEY"] = os.environ.get("FLASK_SECRET_KEY", secrets.token_hex(16))

# Start the app
# On Colab we call python app.py directly
proc = subprocess.Popen(["python", app_py], cwd=BASE, env=env, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Print live output for a short period so user sees startup logs
start_time = time.time()
try:
    while True:
        line = proc.stdout.readline()
        if line:
            print(line, end="")
        # Stop printing after a bit to avoid infinite blocking; user can check logs by keeping the cell running
        if time.time() - start_time > 10:
            print("... (server is running; keep this cell running to keep Flask alive) ...")
            break
except KeyboardInterrupt:
    print("Interrupted, terminating server.")
    proc.terminate()

 * Ngrok URL: https://7a189c81b92f.ngrok-free.app
 * Starting Flask server... (this will run in the background in this cell)
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 910-188-594
127.0.0.1 - - [12/Sep/2025 08:17:51] "GET / HTTP/1.1" 200 -
... (server is running; keep this cell running to keep Flask alive) ...


In [30]:
from google.colab import userdata

# Replace 'YOUR_SECRET_NAME' with the actual name of your secret
my_secret = userdata.get('NGROK_AUTH_TOKEN') # Updated to use NGROK_AUTH_TOKEN

if my_secret:
  print("Secret retrieved successfully!")
  # You can now use my_secret in your code
else:
  print("Secret not found or could not be retrieved. Please ensure the secret is added in Colab's Secrets panel and notebook access is enabled.")

Secret retrieved successfully!
